In [1]:
import numpy as np

In [13]:
m = 40
def verlet_update_position(positions, velocities, forces, dt):
    position_update = []
    for r_i, v_i, f_i in zip(positions, velocities, forces):
        component_update = []
        for i in range(3):
            x_i_dt = r_i[i] + dt*v_i[i] + ((dt**2)/(2*m))*f_i[i]
            component_update.append(x_i_dt)
        position_update.append(component_update)
    return np.array(position_update)

In [6]:
def lennard_jones_potential(distance, epsilon, sigma):
    """
    :returns LJ potential with parameters epsilon, sigma
    """
    return 4 * epsilon * ((sigma ** 12 / distance ** 6) - (sigma ** 6 / distance ** 3))


def calculate_force(delta, r2_ij, epsilon, sigma):
    """
    Calculates the LJ force on a particle in direction delta from another particle
    :param delta: The direction to calculate the force in
    :param r2_ij: Square distance
    :param epsilon: LJ parameter energy
    :param sigma: LJ parameter size
    :return: Pairwise force between two particles
    """
    return delta * (((48 * epsilon) / r2_ij) * ((sigma ** 12 / r2_ij ** 6) - (0.5 * sigma ** 6 / r2_ij ** 3)))


def calculate_configuration_force(coordinates, epsilon, sigma, box_length):
    """
    Given some configuration of particles, calculates the LJ potential/forces pairwise
    :param coordinates: Positions of the particles
    :param epsilon: LJ parameter energy
    :param sigma: LJ parameter size
    :param box_length: Box length to enforce periodic boundary conditions
    :return: The configuration potential, matrix of force on each particle
    """
    potential = 0
    forces = np.zeros((len(coordinates), 3))

    for i, particle_1 in enumerate(coordinates):
        for j in range(i + 1, len(coordinates)):
            particle_2 = coordinates[j]

            # Calculates the delta in each direction and enforce PBC
            delta_x = particle_1[0] - particle_2[0]
            delta_y = particle_1[1] - particle_2[1]
            delta_z = particle_1[2] - particle_2[2]

            delta_x -= box_length * np.round(delta_x / box_length)
            delta_y -= box_length * np.round(delta_y / box_length)
            delta_z -= box_length * np.round(delta_z / box_length)

            r2_ij = delta_x ** 2 + delta_y ** 2 + delta_z ** 2  # the squared distance

            potential_ij = lennard_jones_potential(r2_ij, epsilon, sigma)  # LJ potential
            f_ij_x = calculate_force(delta_x, r2_ij, epsilon, sigma)  # force in each direction
            f_ij_y = calculate_force(delta_y, r2_ij, epsilon, sigma)
            f_ij_z = calculate_force(delta_z, r2_ij, epsilon, sigma)
            f_ij = [f_ij_x, f_ij_y, f_ij_z]

            forces[i, :] = f_ij + forces[i, :]  # Uses N3L to calculate the force on j as - force on i
            forces[j, :] = forces[j, :] - f_ij

            potential += potential_ij

    return potential, forces

In [4]:
positions = np.loadtxt("../../data/init_crds_boxl_3.5-2.dat")
positions

array([[-1.2019639 ,  0.04937449,  1.3446687 ],
       [ 0.95762696,  0.11536136, -1.3533073 ],
       [-0.18897434,  0.52372794, -1.3651418 ],
       [-1.6199141 , -0.04095865,  0.19033262],
       [ 1.5412039 , -0.75625664, -1.0000951 ],
       [-1.1155417 , -1.6664609 , -0.99065462],
       [-0.70326225, -0.13556516, -0.21229811],
       [ 1.0541455 ,  1.3754934 , -1.3764948 ],
       [-0.02434042,  0.09024578,  0.67682947],
       [ 0.57101383, -0.7828554 , -1.0092848 ],
       [-0.75751888,  1.3451685 ,  0.35401673],
       [ 0.02166862,  1.4060785 , -0.56307622],
       [ 1.0072253 , -0.18563138,  0.94703502],
       [-0.17290178, -1.4916186 ,  1.6233682 ],
       [ 0.86533713, -1.2297605 ,  0.93751467],
       [ 1.7441529 , -0.90349879,  1.4845521 ],
       [ 0.22370473, -0.4626245 ,  1.6397821 ],
       [-1.3001678 ,  0.77344676, -1.1569506 ],
       [ 1.6386284 ,  1.270492  , -0.4038093 ],
       [ 0.4666099 ,  0.85132912,  1.231999  ],
       [-0.53585311, -1.3494132 , -0.152

In [8]:
potential, forces = calculate_configuration_force(positions, 0.25, 1, 3.5)
potential

-29.472115017422276

In [9]:
forces

array([[ -2.97837648,   1.99370573,   0.93443728],
       [  1.55677987,   4.26354883,  -0.51921188],
       [ -6.09136609,   3.29395914, -15.77792601],
       [  2.02149539,   5.36256787,   4.26937576],
       [ 10.51332135,  -0.14044396,   3.18859249],
       [ -0.38865308,   1.01391586,  -1.18461974],
       [  1.38356628,  -2.865527  ,  -1.24086902],
       [ -1.60915407,   1.89549032,   3.00377436],
       [  0.03674392,   1.34986474,  -1.96470148],
       [ -8.46651855,  -6.29029246,   9.87245059],
       [ -2.61564248,  -5.17943027,  -2.19099747],
       [  0.2417948 ,  -4.27326044,  -3.81911595],
       [  1.2509662 ,   4.75133719,   5.36481179],
       [  0.91010346,  -2.57999174,   1.89503713],
       [ -1.57809634,  -1.61422931,   1.03976648],
       [  1.27708464,   0.25402929,  -1.96253564],
       [ -2.20455394,   4.10479666,  -6.4870063 ],
       [  0.64906226,  -2.32067543,  -2.20748753],
       [  1.7882653 ,  -0.61876732,   0.1181668 ],
       [-11.6269295 ,   1.81223

In [11]:
v = np.random.standard_normal((len(positions), 3))
v

array([[ 0.48620842,  0.10139854,  0.71957774],
       [-0.46112998,  0.48478951,  0.97915417],
       [-0.7779438 , -0.97318966,  0.73763519],
       [-1.36309273, -0.76634055,  1.63095223],
       [-0.08139057, -0.6735442 ,  1.9940643 ],
       [-0.38707995,  0.93711135,  0.40062969],
       [ 0.66128518, -1.02793606,  0.89929236],
       [ 0.56192165, -0.22724276, -1.43923723],
       [-1.19040585,  0.2811706 ,  0.14459107],
       [-1.68396605,  2.02533798,  0.83927484],
       [ 0.64095247,  1.63816051, -1.42553038],
       [ 0.21444624, -0.24289197,  0.32895921],
       [-1.44098705, -1.23701372,  1.36866016],
       [ 1.81172793,  0.68142506,  0.95257797],
       [ 0.76592277, -0.20337644,  1.58979204],
       [-0.43753378, -0.3135057 ,  1.42664987],
       [-1.60129517,  1.15185553, -0.79202087],
       [ 1.69488147,  0.38584501,  0.92457205],
       [-1.1319444 , -0.51199272,  0.72658988],
       [-1.01463366, -0.64875229,  0.62906894],
       [-1.40249261,  2.56828924,  0.033

In [14]:
verlet_update_position(positions, v, forces, 0.005)

array([[-1.19953379,  0.04988211,  1.34826688],
       [ 0.9553218 ,  0.11778664, -1.34841169],
       [-0.19286596,  0.51886302, -1.36145855],
       [-1.62672893, -0.04478867,  0.19848872],
       [ 1.54080023, -0.7596244 , -0.99012378],
       [-1.11747722, -1.66177503, -0.98865184],
       [-0.69995539, -0.14070574, -0.20780204],
       [ 1.05695461,  1.37435778, -1.38369005],
       [-0.03029244,  0.09165205,  0.67755181],
       [ 0.56259135, -0.77273068, -1.00508534],
       [-0.75431494,  1.35335768,  0.34688839],
       [ 0.02274092,  1.4048627 , -0.56143262],
       [ 1.00002076, -0.19181496,  0.95388   ],
       [-0.16384286, -1.48821228,  1.62813168],
       [ 0.86916625, -1.23077789,  0.94546396],
       [ 1.74196563, -0.90506624,  1.49168474],
       [ 0.21569757, -0.45686394,  1.63581997],
       [-1.29169319,  0.77537526, -1.15232843],
       [ 1.63296924,  1.26793184, -0.40017631],
       [ 0.4615331 ,  0.84808592,  1.2351442 ],
       [-0.54286532, -1.33656792, -0.152